# Treelets Python Implementation

Python implementation of the treelets algorithm to cement my understanding. Working functions will be wrapped in a module and used to compare the treelets decomposition to PCA. 

## Setup

In [ ]:
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import sys
import numpy as np
import pandas as pd
import os

In [ ]:
sys.path.append("../scripts")
import python_treelet_implementation as pytree
import utils

## Workings

Example of the implementation applied to noise. Note that both original paper and CRAN implementation use the correlation between two variables a measure of their similarity. The parameter `abs_` controls whether the correlation or absolute correlation is used as a similarity measure. 

In [ ]:
test_data = np.array([np.random.normal(0,1,400)])\
              .reshape(4,100)
np.cov(test_data)

Build treelet decomposition.  

In [ ]:
tree = pytree.treelet_decomposition(X = test_data, 
                                    L = 4)

### Treelet covariance matrix

At each level we can extract the treelet variance-covaraince matrix.  

In [ ]:
tree[2]["C"]

### Orderings

We can extract which variables were merged at each level, and which was designated the sum variable. 

In [ ]:
tree[2]["pair"]

In [ ]:
tree[2]["order"]

### Dirac basis 

At each level we can extract the Dirac basis. 

In [ ]:
tree[2]["B"]

## Comparison with CRAN test data

Python implementation applied to similarity matrix supplied with the package. Prior to applying the treelet transform no block structure is apparent.

In [ ]:
Ahat = pd.read_csv("../data/Ahat.csv")\
         .drop("Unnamed: 0", axis = 1)\
         .to_numpy()
plt.matshow(Ahat)

The implementation achieves the same treelet covaraince matrix (up to rounding errors) and varaible merges in the same order as code on CRAN. Even at low levels of the tree the block structure is apparent. Using the absolute value of the correlation matrix lead to variable merges being performed a different order, and a different treelet decomposition. At low levels a block structure is still visible. 

In [ ]:
tree = pytree.treelet_decomposition(X = Ahat,
                                    L = 50)
plt.matshow(tree[10]["C"])

## Comparison with Theory on Block Covaraince Matrices

### Test with Lemma 1

At each level $l$ the population treelet decomposition on single-block covaraince matrix returns sum varaible $s_{l,i}$ which is defined on a disjoint set $\mathcal{A}_{l,i} \subseteq \left \{ 1,...,p \right \}$ and a scaling function $\phi_{l,i}$ with non-zero entries only for elements in $\mathcal{A}_{l,i}$, in particular: 

* sum variable: $s_{l,i} = \frac{1}{\sqrt{\mathcal{A}_{l,i}}} \sum_{j \in \mathcal{A}_{l,i}} x_j$
* scaling function: $\phi_{l,i} = \frac{1}{\sqrt{\mathcal{A}_{l,i}}}\times \mathcal{I}_{s_{l,i}}$

The following example with a $5 \times 5$ matrix of ones shows this to be true. At the population level additive noised corresponds to adding the noise variance to the diagonal of the correlation matrix. 

In [ ]:
tree = pytree.treelet_decomposition(X = np.ones((5,5)) + .2*np.eye(5),
                                    L = 5)

In [ ]:
tree[4]["B"]

### Tests with Theorem 2

The population treelet decomposition on a standard block covariance matrix having $k$ blocks with low "between block" covariance and high "within block" covaraince returns sum variables and scaling functions which are constant on each block $\mathcal{B}_k$, in particuar: 

* sum variable: $s_k = \frac{1}{\sqrt{p_k}}\sum_{j \in \mathcal{B}_k} x_k$
* scaling function: $\phi_{k} = \frac{1}{\sqrt{p_k}}\times\mathcal{I}_{\mathcal{B}_k}$

The following example with a block diagonal matrix shows this to be true. 

In [ ]:
A = B = C = np.ones((2,2))
tree = pytree.treelet_decomposition(X = block_diag(A,B,C) + .2*np.eye(6),
                                    L = 6
                                   )

In [ ]:
tree[3]["B"]